# Ejercicio de BYTE-PAIR ENCODING
Integrantes:
*   Aguilar Valenzuela Luis Hector
*   Camargo Loaiza Julio Andres
*   Minjares Neriz Victor Manuel

In [3]:
import re, collections

### get_vocab(filename)
Función que recibe el nombre de el archivo de texto y devuelve un vocabulario de palabras con la frecuencia de cada palabra y un separador en cada palabra

# Codigo principal

In [ ]:
# Minicorpus a usar :
# https://drive.google.com/file/d/17h_rLrWL2xg3jD0U1CCseeaAd6t17yc0/view?usp=share_linket 

vocab = get_vocab('miniCorpus.txt')

print('==========')
print('Tokens Before BPE')
tokens = get_tokens(vocab)
print('Tokens: {}'.format(tokens))
print('Number of tokens: {}'.format(len(tokens)))
print('==========')

num_merges = 15
for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print('Iter: {}'.format(i))
    print('Best pair: {}'.format(best))
    tokens = get_tokens(vocab)
    print('Tokens: {}'.format(tokens))
    print('Number of tokens: {}'.format(len(tokens)))
    print('==========')